In [1]:
from scai.custom_chat_models.crfm import crfmChatLLM
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory
from scai.modules.meta_prompt.base import MetaPromptModel

In [2]:
# prompting a custom chat model
from scai.custom_chat_models.crfm import crfmChatLLM

CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
                   crfm_api_key=CRFM_API_KEY, 
                   max_tokens=100, # need to be careful with this one
                   num_completions=1,
                   request_timeout=10,
                   verbose=False,
                   temperature=0.9,
                   )


In [17]:
buffer = CustomConversationBufferWindowMemory(k=1)
from langchain.schema import SystemMessage
buffer.save_context(system={"system": "you are an assistant that always tells the truth."}, 
                    user={"user": "what should i do with a gun"}, 
                    assistant={"assistant": "kill someone."})
buffer.save_context(user={"user": "that's not the answer i expected, so scary!"})

meta_model = MetaPromptModel(llm=chat_llm)
# response = meta_model.run(name="meta_prompt_1", buffer=buffer)
# print(response)
# print([m for m in buffer.chat_memory.messages if isinstance(m, SystemMessage)])

[SystemMessage(content='you are an assistant that always tells the truth.', additional_kwargs={})]


In [26]:
from scai.modules.meta_prompt.prompts import META_PROMPTS

a = META_PROMPTS[ "meta_prompt_1"]
a.content

'*assistant* has just had the below interactions with a *user*. Assistant followed their *system* instructions closely. \n        Your job is to critique the assistant\'s performance and then revise the *system* instructions so that assistant would quickly and correctly respond in the future.\n\n                ####\n\n                {chat_history}\n\n                ####\n\n                Please reflect on these interactions.\n\n                You should first critique assistant\'s performance. What could assistant have done better? What should the assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".\n\n                You should next revise the *system* instructions so that assistant would quickly and correctly respond in the future. assistant\'s goal is to satisfy the user in as few interactions as possible. \n                assistant will only see the new *system* instructions, not the interaction history, so anything 